# Heatmap Visualization Tutorial: Failure Transition Matrices

## Learning Objectives

By completing this tutorial, you will be able to:
- ✅ Load and explore labeled failure trace data
- ✅ Extract failure transitions from trace records
- ✅ Build transition count matrices programmatically
- ✅ Configure seaborn heatmaps for effective visualization
- ✅ Interpret heatmap patterns to identify bottlenecks
- ✅ Export publication-quality heatmap visualizations
- ✅ Perform basic bottleneck analysis from matrices

## Prerequisites

- Completed [Transition Analysis Concepts](transition_analysis_concepts.md)
- Understanding of Python dictionaries and lists
- Familiarity with pandas and matplotlib

## Estimated Time

**Execution Time:** 15-20 minutes  
**Hands-on Exercises:** 10-15 minutes

## Setup and Imports

In [ ]:
# Standard library
import json
from pathlib import Path
from collections import Counter
from typing import Dict, List, Tuple

# Data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure visualization defaults
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (10, 8)

print("\u2713 Imports successful")

## 1. Load Labeled Trace Data

We'll load the pre-labeled failure traces from `data/labeled_traces.json`. Each trace contains:
- `trace_id`: Unique identifier
- `last_success_state`: Final state that succeeded
- `first_failure_state`: First state that failed
- `messages`: Full conversation history
- Additional metadata

In [ ]:
# Define file paths
DATA_FILE = Path("data/labeled_traces.json")

# Load traces
if not DATA_FILE.exists():
    raise FileNotFoundError(
        f"Data file not found: {DATA_FILE}\n"
        "Make sure you're running this notebook from the homeworks/hw5/ directory."
    )

with open(DATA_FILE, 'r') as f:
    traces = json.load(f)

print(f"✓ Loaded {len(traces)} labeled failure traces")
print(f"✓ File: {DATA_FILE.resolve()}")

### Explore a Sample Trace

Let's examine the structure of a single trace to understand what data we're working with.

In [ ]:
# Display first trace (pretty-printed)
sample_trace = traces[0]

print("Sample Trace Structure:")
print("=" * 60)
print(f"Trace ID: {sample_trace['trace_id']}")
print(f"Customer Persona: {sample_trace.get('customer_persona', 'N/A')}")
print(f"Last Success State: {sample_trace['last_success_state']}")
print(f"First Failure State: {sample_trace['first_failure_state']}")
print(f"Number of Messages: {len(sample_trace.get('messages', []))}")
print("\nFailure Transition:")
print(f"  {sample_trace['last_success_state']} → {sample_trace['first_failure_state']}")
print("=" * 60)

### Define Pipeline States

The recipe bot follows a 10-state pipeline. We'll define these states in order to structure our matrix.

In [ ]:
# 10-state pipeline (in execution order)
PIPELINE_STATES = [
    "ParseRequest",        # LLM: Parse user query
    "PlanToolCalls",       # LLM: Decide which tools to use
    "GenCustomerArgs",     # LLM: Generate customer DB arguments
    "GetCustomerProfile",  # TOOL: Fetch customer profile
    "GenRecipeArgs",       # LLM: Generate recipe search arguments
    "GetRecipes",          # TOOL: Search recipe database
    "GenWebArgs",          # LLM: Generate web search arguments
    "GetWebInfo",          # TOOL: Perform web search
    "ComposeResponse",     # LLM: Draft final response
    "DeliverResponse",     # SYSTEM: Send response to user
]

# Create state → index mapping
STATE_INDEX = {state: idx for idx, state in enumerate(PIPELINE_STATES)}

print(f"✓ Pipeline has {len(PIPELINE_STATES)} states")
print("\nState Types:")
print("  LLM States:   ParseRequest, PlanToolCalls, GenCustomerArgs, GenRecipeArgs, GenWebArgs, ComposeResponse")
print("  Tool States:  GetCustomerProfile, GetRecipes, GetWebInfo")
print("  System State: DeliverResponse")

## 2. Extract Failure Transitions

A **failure transition** is the pair `(last_success_state, first_failure_state)` for each trace.

Let's extract all transitions and count their frequencies.

In [ ]:
def extract_failure_transitions(traces: List[Dict]) -> List[Tuple[str, str]]:
    """Extract all (last_success, first_failure) pairs from traces.
    
    Args:
        traces: List of labeled trace dictionaries
        
    Returns:
        List of (from_state, to_state) tuples
    """
    transitions = []
    
    for trace in traces:
        from_state = trace.get('last_success_state')
        to_state = trace.get('first_failure_state')
        
        # Validate both states exist and are in our pipeline
        if from_state in STATE_INDEX and to_state in STATE_INDEX:
            transitions.append((from_state, to_state))
        else:
            print(f"⚠ Skipped invalid transition in {trace.get('trace_id')}: {from_state} → {to_state}")
    
    return transitions

# Extract transitions
failure_transitions = extract_failure_transitions(traces)

print(f"✓ Extracted {len(failure_transitions)} failure transitions")
print(f"✓ Coverage: {len(failure_transitions)}/{len(traces)} traces ({len(failure_transitions)/len(traces)*100:.1f}%)")

### Count Transition Frequencies

Use `Counter` to tally how many times each transition occurs.

In [ ]:
# Count transition frequencies
transition_counts = Counter(failure_transitions)

print(f"Total unique transitions: {len(transition_counts)}")
print(f"\nTop 5 Most Common Failure Transitions:")
print("=" * 70)

for (from_state, to_state), count in transition_counts.most_common(5):
    percentage = count / len(failure_transitions) * 100
    print(f"{from_state:20} → {to_state:20} | Count: {count:3} ({percentage:5.1f}%)")

print("=" * 70)

## 3. Build Transition Matrix

Convert the transition counts into a 10×10 matrix where:
- **Rows** = Last successful state
- **Columns** = First failing state
- **Cell value** = Number of times that transition occurred

In [ ]:
def build_transition_matrix(traces: List[Dict], states: List[str]) -> np.ndarray:
    """Build transition count matrix from labeled traces.
    
    Args:
        traces: List of labeled trace dictionaries
        states: Ordered list of pipeline states
        
    Returns:
        n×n numpy array where n = len(states)
    """
    n = len(states)
    matrix = np.zeros((n, n), dtype=int)
    state_to_idx = {s: i for i, s in enumerate(states)}
    
    for trace in traces:
        from_state = trace.get('last_success_state')
        to_state = trace.get('first_failure_state')
        
        # Only count valid transitions
        if from_state in state_to_idx and to_state in state_to_idx:
            i = state_to_idx[from_state]
            j = state_to_idx[to_state]
            matrix[i, j] += 1
    
    return matrix

# Build matrix
transition_matrix = build_transition_matrix(traces, PIPELINE_STATES)

print(f"✓ Built {transition_matrix.shape[0]}×{transition_matrix.shape[1]} transition matrix")
print(f"✓ Total failures recorded: {transition_matrix.sum():.0f}")
print(f"✓ Non-zero cells: {np.count_nonzero(transition_matrix)}/{transition_matrix.size}")

### View Matrix as DataFrame

Convert to pandas DataFrame for easier inspection.

In [ ]:
# Create labeled DataFrame
matrix_df = pd.DataFrame(
    transition_matrix,
    index=PIPELINE_STATES,
    columns=PIPELINE_STATES
)

print("Transition Matrix (rows=last success, columns=first failure):")
print("=" * 100)
display(matrix_df)
print("=" * 100)

## 4. Visualize with Seaborn Heatmap

Create an annotated heatmap to visualize failure patterns.

In [ ]:
def plot_transition_heatmap(
    matrix: np.ndarray,
    states: List[str],
    title: str = "Failure Transition Heatmap",
    cmap: str = "YlOrRd",
    figsize: Tuple[int, int] = (12, 10)
) -> None:
    """Plot annotated heatmap of transition matrix.
    
    Args:
        matrix: Transition count matrix
        states: State labels for axes
        title: Plot title
        cmap: Seaborn color palette
        figsize: Figure dimensions (width, height)
    """
    plt.figure(figsize=figsize)
    
    # Create heatmap
    sns.heatmap(
        matrix,
        annot=True,           # Show count values in cells
        fmt="d",              # Integer format
        cmap=cmap,            # Color scheme (yellow → red)
        xticklabels=states,
        yticklabels=states,
        cbar_kws={"label": "Failure Count"},
        square=True,          # Square cells
        linewidths=0.5,       # Grid lines
        linecolor='gray'
    )
    
    # Formatting
    plt.title(title, fontsize=16, fontweight='bold', pad=20)
    plt.xlabel("First Failure State →", fontsize=12, fontweight='bold')
    plt.ylabel("Last Success State ↓", fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

# Generate heatmap
plot_transition_heatmap(transition_matrix, PIPELINE_STATES)

### Interpret the Heatmap

**Reading the Heatmap:**
- **Darker cells** = Higher failure counts
- **Row** = Where the system last succeeded
- **Column** = Where the system first failed
- **Diagonal** = State succeeded, then immediately failed on retry (rare)
- **Off-diagonal** = Failure in next sequential state (common)

**Common Patterns:**
1. **Single dark cell** = Clustered bottleneck (most failures at one transition)
2. **Dark column** = Specific state fails from multiple predecessors
3. **Dark row** = After one state, failures scatter across next states
4. **Distributed darkness** = No clear bottleneck (diverse failures)

## 5. Bottleneck Analysis

Perform quantitative analysis to identify system bottlenecks.

### Which States Fail Most Often? (Column Sums)

In [ ]:
# Sum failures by column (how often each state fails)
column_sums = transition_matrix.sum(axis=0)

# Create summary DataFrame
failure_by_state = pd.DataFrame({
    'State': PIPELINE_STATES,
    'Total Failures': column_sums,
    'Percentage': column_sums / column_sums.sum() * 100
}).sort_values('Total Failures', ascending=False)

print("States Ranked by Failure Frequency (Column Sums):")
print("=" * 60)
display(failure_by_state)
print("=" * 60)

# Identify primary bottleneck
primary_bottleneck = failure_by_state.iloc[0]
print(f"\n🔴 PRIMARY BOTTLENECK: {primary_bottleneck['State']}")
print(f"   Fails {primary_bottleneck['Total Failures']:.0f} times ({primary_bottleneck['Percentage']:.1f}% of all failures)")

### After Which States Do Failures Occur? (Row Sums)

In [ ]:
# Sum failures by row (after which state do failures happen)
row_sums = transition_matrix.sum(axis=1)

# Create summary DataFrame
failures_after_state = pd.DataFrame({
    'State': PIPELINE_STATES,
    'Failures After This State': row_sums,
    'Percentage': row_sums / row_sums.sum() * 100
}).sort_values('Failures After This State', ascending=False)

print("States with Most Downstream Failures (Row Sums):")
print("=" * 70)
display(failures_after_state)
print("=" * 70)

# Identify upstream cause
upstream_issue = failures_after_state.iloc[0]
print(f"\n⚠️  UPSTREAM RISK: {upstream_issue['State']}")
print(f"   Followed by {upstream_issue['Failures After This State']:.0f} failures ({upstream_issue['Percentage']:.1f}% of total)")

### Most Common Single Transition

In [ ]:
# Find cell with maximum value
max_value = transition_matrix.max()
max_indices = np.argwhere(transition_matrix == max_value)

print(f"Most Frequent Failure Transition(s):")
print("=" * 70)

for i, j in max_indices:
    from_state = PIPELINE_STATES[i]
    to_state = PIPELINE_STATES[j]
    percentage = max_value / transition_matrix.sum() * 100
    
    print(f"  {from_state} → {to_state}")
    print(f"  Count: {max_value} occurrences ({percentage:.1f}% of all failures)")
    print()

print("=" * 70)

## 6. Export Heatmap

Save publication-quality PNG for reports.

In [ ]:
def save_heatmap(
    matrix: np.ndarray,
    states: List[str],
    output_path: Path,
    dpi: int = 300
) -> None:
    """Save heatmap to PNG file.
    
    Args:
        matrix: Transition count matrix
        states: State labels
        output_path: Output file path
        dpi: Resolution (300 for publication quality)
    """
    # Ensure output directory exists
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # Create figure
    plt.figure(figsize=(12, 10))
    sns.heatmap(
        matrix,
        annot=True,
        fmt="d",
        cmap="YlOrRd",
        xticklabels=states,
        yticklabels=states,
        cbar_kws={"label": "Failure Count"},
        square=True,
        linewidths=0.5,
        linecolor='gray'
    )
    
    plt.title("Failure Transition Heatmap", fontsize=16, fontweight='bold', pad=20)
    plt.xlabel("First Failure State →", fontsize=12, fontweight='bold')
    plt.ylabel("Last Success State ↓", fontsize=12, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    
    # Save
    plt.savefig(output_path, dpi=dpi, bbox_inches='tight')
    plt.close()
    
    print(f"✓ Heatmap saved to: {output_path.resolve()}")
    print(f"  Resolution: {dpi} DPI")
    print(f"  File size: {output_path.stat().st_size / 1024:.1f} KB")

# Save to results/
OUTPUT_PATH = Path("results/failure_transition_heatmap.png")
save_heatmap(transition_matrix, PIPELINE_STATES, OUTPUT_PATH)

## 7. Exercises and Next Steps

### Exercise 1: Filter by Customer Persona

Build separate transition matrices for different customer personas (e.g., `vegan_family`, `gluten_free_athlete`) to see if failure patterns differ.

In [ ]:
# TODO: Group traces by customer_persona and build separate matrices
# Hint: Use a dictionary comprehension to group traces

# Example starter code:
# personas = set(t.get('customer_persona') for t in traces if t.get('customer_persona'))
# for persona in personas:
#     persona_traces = [t for t in traces if t.get('customer_persona') == persona]
#     persona_matrix = build_transition_matrix(persona_traces, PIPELINE_STATES)
#     plot_transition_heatmap(persona_matrix, PIPELINE_STATES, title=f"Failures: {persona}")

### Exercise 2: Investigate High-Frequency Transition

Read all traces with the most common failure transition to identify patterns.

In [ ]:
# TODO: Filter traces by (last_success_state, first_failure_state) pair
# Examine their user queries, customer personas, or other metadata

# Example starter code:
# target_from = "GenRecipeArgs"
# target_to = "GetRecipes"
# matching_traces = [
#     t for t in traces 
#     if t['last_success_state'] == target_from and t['first_failure_state'] == target_to
# ]
# print(f"Found {len(matching_traces)} traces with {target_from} → {target_to}")
# # Inspect first few
# for trace in matching_traces[:3]:
#     print(trace['trace_id'], trace.get('customer_persona'))

### Exercise 3: Normalized Heatmap (Percentages)

Create a heatmap showing percentages instead of raw counts.

In [ ]:
# TODO: Normalize matrix to percentages
# Hint: normalized_matrix = (transition_matrix / transition_matrix.sum()) * 100

# Then plot with fmt=".1f" to show 1 decimal place

## Summary and Key Takeaways

In this tutorial, you learned to:

1. **Load and explore labeled failure traces** from JSON
2. **Extract failure transitions** as `(last_success, first_failure)` pairs
3. **Build transition count matrices** programmatically
4. **Visualize patterns with seaborn heatmaps** using effective color schemes
5. **Identify bottlenecks** through column/row sum analysis
6. **Export publication-quality visualizations** at 300 DPI

### Next Steps

- **Read traces** with high-frequency transitions to understand root causes
- **Propose fixes** for identified bottlenecks (better prompts, tool improvements)
- **Re-run analysis** after implementing fixes to verify improvement
- **Explore** [Transition Analysis Concepts](transition_analysis_concepts.md) for deeper methodology

### Related Tutorials

- [Transition Analysis Concepts](transition_analysis_concepts.md) - Theory and methodology
- [Transition Matrix Diagram](diagrams/transition_matrix_concept.mmd) - Visual pipeline reference
- [HW5 Tutorial Index](TUTORIAL_INDEX.md) - Complete learning path

---

**Tutorial Status:** ✅ Complete  
**Last Updated:** 2025-10-30  
**Maintainer:** AI Evaluation Course Team